<a href="https://colab.research.google.com/github/Anjasfedo/Code-as-a-Cryptography/blob/main/ecc_lsb_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ECC

In [1]:
!pip install eciespy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.5/100.5 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 55.3 MB/s eta 0:00:00


# LSB

In [2]:
!git clone https://github.com/RobinDavid/LSB-Steganography && cd LSB-Steganography && ls && pip install -r requirements.txt

Cloning into 'LSB-Steganography'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 75 (delta 1), reused 2 (delta 0), pack-reused 68 (from 1)
Receiving objects: 100% (75/75), 19.72 KiB | 9.86 MiB/s, done.
Resolving deltas: 100% (26/26), done.
LICENCE  LSBSteg.py  README.md	requirements.txt
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13704 sha256=cfb308100eaea315aece6778f344a78d8e6dfb4f18d41d0cf5c5c950381ee4a9
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [3]:
#!/usr/bin/env python
# coding:UTF-8
"""LSBSteg.py

Usage:
  LSBSteg.py encode -i <input> -o <output> -f <file>
  LSBSteg.py decode -i <input> -o <output>

Options:
  -h, --help                Show this help
  --version                 Show the version
  -f,--file=<file>          File to hide
  -i,--in=<input>           Input image (carrier)
  -o,--out=<output>         Output image (or extracted file)
"""

import cv2
import docopt
import numpy as np


class SteganographyException(Exception):
    pass


class LSBSteg():
    def __init__(self, im):
        self.image = im
        self.height, self.width, self.nbchannels = im.shape
        self.size = self.width * self.height

        self.maskONEValues = [1,2,4,8,16,32,64,128]
        #Mask used to put one ex:1->00000001, 2->00000010 .. associated with OR bitwise
        self.maskONE = self.maskONEValues.pop(0) #Will be used to do bitwise operations

        self.maskZEROValues = [254,253,251,247,239,223,191,127]
        #Mak used to put zero ex:254->11111110, 253->11111101 .. associated with AND bitwise
        self.maskZERO = self.maskZEROValues.pop(0)

        self.curwidth = 0  # Current width position
        self.curheight = 0 # Current height position
        self.curchan = 0   # Current channel position

    def put_binary_value(self, bits): #Put the bits in the image
        for c in bits:
            val = list(self.image[self.curheight,self.curwidth]) #Get the pixel value as a list
            if int(c) == 1:
                val[self.curchan] = int(val[self.curchan]) | self.maskONE #OR with maskONE
            else:
                val[self.curchan] = int(val[self.curchan]) & self.maskZERO #AND with maskZERO

            self.image[self.curheight,self.curwidth] = tuple(val)
            self.next_slot() #Move "cursor" to the next space

    def next_slot(self):#Move to the next slot were information can be taken or put
        if self.curchan == self.nbchannels-1: #Next Space is the following channel
            self.curchan = 0
            if self.curwidth == self.width-1: #Or the first channel of the next pixel of the same line
                self.curwidth = 0
                if self.curheight == self.height-1:#Or the first channel of the first pixel of the next line
                    self.curheight = 0
                    if self.maskONE == 128: #Mask 1000000, so the last mask
                        raise SteganographyException("No available slot remaining (image filled)")
                    else: #Or instead of using the first bit start using the second and so on..
                        self.maskONE = self.maskONEValues.pop(0)
                        self.maskZERO = self.maskZEROValues.pop(0)
                else:
                    self.curheight +=1
            else:
                self.curwidth +=1
        else:
            self.curchan +=1

    def read_bit(self): #Read a single bit int the image
        val = self.image[self.curheight,self.curwidth][self.curchan]
        val = int(val) & self.maskONE
        self.next_slot()
        if val > 0:
            return "1"
        else:
            return "0"

    def read_byte(self):
        return self.read_bits(8)

    def read_bits(self, nb): #Read the given number of bits
        bits = ""
        for i in range(nb):
            bits += self.read_bit()
        return bits

    def byteValue(self, val):
        return self.binary_value(val, 8)

    def binary_value(self, val, bitsize): #Return the binary value of an int as a byte
        binval = bin(val)[2:]
        if len(binval) > bitsize:
            raise SteganographyException("binary value larger than the expected size")
        while len(binval) < bitsize:
            binval = "0"+binval
        return binval

    def encode_text(self, txt):
        l = len(txt)
        binl = self.binary_value(l, 16) #Length coded on 2 bytes so the text size can be up to 65536 bytes long
        self.put_binary_value(binl) #Put text length coded on 4 bytes
        for char in txt: #And put all the chars
            c = ord(char)
            self.put_binary_value(self.byteValue(c))
        return self.image

    def decode_text(self):
        ls = self.read_bits(16) #Read the text size in bytes
        l = int(ls,2)
        i = 0
        unhideTxt = ""
        while i < l: #Read all bytes of the text
            tmp = self.read_byte() #So one byte
            i += 1
            unhideTxt += chr(int(tmp,2)) #Every chars concatenated to str
        return unhideTxt

    def encode_image(self, imtohide):
        w = imtohide.width
        h = imtohide.height
        if self.width*self.height*self.nbchannels < w*h*imtohide.channels:
            raise SteganographyException("Carrier image not big enough to hold all the datas to steganography")
        binw = self.binary_value(w, 16) #Width coded on to byte so width up to 65536
        binh = self.binary_value(h, 16)
        self.put_binary_value(binw) #Put width
        self.put_binary_value(binh) #Put height
        for h in range(imtohide.height): #Iterate the hole image to put every pixel values
            for w in range(imtohide.width):
                for chan in range(imtohide.channels):
                    val = imtohide[h,w][chan]
                    self.put_binary_value(self.byteValue(int(val)))
        return self.image


    def decode_image(self):
        width = int(self.read_bits(16),2) #Read 16bits and convert it in int
        height = int(self.read_bits(16),2)
        unhideimg = np.zeros((width,height, 3), np.uint8) #Create an image in which we will put all the pixels read
        for h in range(height):
            for w in range(width):
                for chan in range(unhideimg.channels):
                    val = list(unhideimg[h,w])
                    val[chan] = int(self.read_byte(),2) #Read the value
                    unhideimg[h,w] = tuple(val)
        return unhideimg

    def encode_binary(self, data):
        l = len(data)
        if self.width*self.height*self.nbchannels < l+64:
            raise SteganographyException("Carrier image not big enough to hold all the datas to steganography")
        self.put_binary_value(self.binary_value(l, 64))
        for byte in data:
            byte = byte if isinstance(byte, int) else ord(byte) # Compat py2/py3
            self.put_binary_value(self.byteValue(byte))
        return self.image

    def decode_binary(self):
        l = int(self.read_bits(64), 2)
        output = b""
        for i in range(l):
            output += bytearray([int(self.read_byte(),2)])
        return output


def main():
    args = docopt.docopt(__doc__, version="0.2")
    in_f = args["--in"]
    out_f = args["--out"]
    in_img = cv2.imread(in_f)
    steg = LSBSteg(in_img)
    lossy_formats = ["jpeg", "jpg"]

    if args['encode']:
        #Handling lossy format
        out_f, out_ext = out_f.split(".")
        if out_ext in lossy_formats:
            out_f = out_f + ".png"
            print("Output file changed to ", out_f)

        data = open(args["--file"], "rb").read()
        res = steg.encode_binary(data)
        cv2.imwrite(out_f, res)

    elif args["decode"]:
        raw = steg.decode_binary()
        with open(out_f, "wb") as f:
            f.write(raw)

# if __name__=="__main__":
#     main()

In [42]:
LENA_IMG = 'ori-image/lena.png'

In [74]:
def embed_lsb(target, content, ori_img = LENA_IMG):
  steg = LSBSteg(cv2.imread(ori_img))

  img_encoded = steg.encode_text(content)

  lsb_img = f"{stego_dir_img}/{target}"

  cv2.imwrite(lsb_img, img_encoded)

In [99]:
def extract_lsb(target):
  im = cv2.imread(f"{stego_dir_img}/{target}")

  steg = LSBSteg(im)

  content = steg.decode_text()

  return content

# PSNR

In [78]:
def calculate_psnr_2(target, ori_image = LENA_IMG):
    # Read images
    original = cv2.imread(ori_image)
    stego = cv2.imread(f'{stego_dir_img}/{target}')

    if original is None:
        raise ValueError(f"Failed to load original image from {ori_image}. Ensure the file exists and is a valid image format.")
    if stego is None:
        raise ValueError(f"Failed to load stego image from {stego_dir_img}/{target}. Ensure the file exists and is a valid image format.")

    # Calculate MSE (Mean Squared Error)
    mse = np.mean((original - stego) ** 2)
    if mse == 0:  # If images are identical
        return float('inf')

    # Calculate PSNR using the 10 log10 version
    max_pixel_value = 255.0
    psnr = 10 * np.log10((max_pixel_value ** 2) / mse)
    print(f'PSNR: {psnr}')
    return psnr

# Test Image

In [45]:
import os
# Directory where the image will be saved
ori_dir_img = "ori-image"

# Create the ori_dir_img if it doesn't exist
if not os.path.exists(ori_dir_img):
    os.makedirs(ori_dir_img)

stego_dir_img = "stego-image"

# Create the stego_dir_img if it doesn't exist
if not os.path.exists(stego_dir_img):
    os.makedirs(stego_dir_img)

## Lena Image

In [46]:
import requests
from PIL import Image
import io

# URL to the raw image file
url = "https://raw.githubusercontent.com/mikolalysenko/lena/master/lena.png"

# Download the image
response = requests.get(url)
if response.status_code == 200:
    # Load the image using PIL
    lena_image = Image.open(io.BytesIO(response.content))
    lena_image.show()  # Display the image (optional)
    lena_image.save("ori-image/lena.png")  # Save the image locally
else:
    print("Failed to download the image.")

In [47]:
LENA_IMG = 'ori-image/lena.png'

if not os.path.exists(LENA_IMG):
    raise FileNotFoundError(f"Image not found at {LENA_IMG}")

# LSB Only

In [85]:
CONTENT = "3348610401970005#christofer*derian*budianto#tegal#1997-03-04#laki-laki#b#jl.*pala*22*no.*30#005#017#mejasem*tengah#kramat#katholik#belum*kawin#pelajar/mahasiswa#wni#seumur*hidup"

embed_lsb("lsb_only.png", CONTENT)

In [86]:
extract_lsb("lsb_only.png")

3348610401970005#christofer*derian*budianto#tegal#1997-03-04#laki-laki#b#jl.*pala*22*no.*30#005#017#mejasem*tengah#kramat#katholik#belum*kawin#pelajar/mahasiswa#wni#seumur*hidup


'3348610401970005#christofer*derian*budianto#tegal#1997-03-04#laki-laki#b#jl.*pala*22*no.*30#005#017#mejasem*tengah#kramat#katholik#belum*kawin#pelajar/mahasiswa#wni#seumur*hidup'

In [87]:
calculate_psnr_2("lsb_only.png")

PSNR: 78.57483188820159


78.57483188820159

# LSB with ECC

In [93]:
from ecies.utils import generate_eth_key
from ecies import encrypt, decrypt

private_key = generate_eth_key()
public_key = private_key.public_key

public_key_hex = public_key.to_hex()
private_key_hex = private_key.to_hex()

print("Public Key:", public_key_hex)
print("Private Key:", private_key_hex)

ciphertext = encrypt(public_key_hex, CONTENT.encode())

print("Ciphertext:", ciphertext.hex())

decrypted_message = decrypt(private_key_hex, ciphertext)
print("Decrypted Message:", decrypted_message.decode())

Public Key: 0x68bf17893f3e43df538fbcc1d03cfff542ea6ed4bd206c49d03fe5557570ec1dcbab9714b174524269e2de6054764340ef704ee00183d823c394840f12365006
Private Key: 0x71ff1eacc7fb45e7ead8539cbdadd647cb8fb24f711d9db1277e60dc8af2800f
Ciphertext: 04c940f89e4b2fb757056cc643161db1753c9d16fb614d0b7e4a77a6bbe3fccb57d4af9b218fc98cfb3d26ed98d64b285c6440609479eecdfbc87e21b6edbdead947d6f09a34961aa50f8ca6e726ff0a73844e16f67cdf8ccff1339ab76923d5678c7e004d8461d3af6bea7b89bf0fbfa5b2214aeb43b288919e7c565463bb677b02aeb2619763622a4707cbac1460e3b8d118b243890229f450991da0907dd4c8c28bc645078ce082748f90563ffb7f3c23d973c3a3f7ea957631fa249e5193fde71c2241a801ea6899c1a18caa27ec715a218ec9a6c77eff9c798bc2a5dba6f8444dff711fe5104afe27a30c0b8b2228266561794715cceaf186bc04d6b55e0cdf3557d28ce83d9907eeb97f4129874e45
Decrypted Message: 3348610401970005#christofer*derian*budianto#tegal#1997-03-04#laki-laki#b#jl.*pala*22*no.*30#005#017#mejasem*tengah#kramat#katholik#belum*kawin#pelajar/mahasiswa#wni#seumur*hidup


In [95]:
print(ciphertext)
print(ciphertext.hex())

b'\x04\xc9@\xf8\x9eK/\xb7W\x05l\xc6C\x16\x1d\xb1u<\x9d\x16\xfbaM\x0b~Jw\xa6\xbb\xe3\xfc\xcbW\xd4\xaf\x9b!\x8f\xc9\x8c\xfb=&\xed\x98\xd6K(\\d@`\x94y\xee\xcd\xfb\xc8~!\xb6\xed\xbd\xea\xd9G\xd6\xf0\x9a4\x96\x1a\xa5\x0f\x8c\xa6\xe7&\xff\ns\x84N\x16\xf6|\xdf\x8c\xcf\xf13\x9a\xb7i#\xd5g\x8c~\x00M\x84a\xd3\xafk\xea{\x89\xbf\x0f\xbf\xa5\xb2!J\xebC\xb2\x88\x91\x9e|VTc\xbbg{\x02\xae\xb2a\x97cb*G\x07\xcb\xac\x14`\xe3\xb8\xd1\x18\xb2C\x89\x02)\xf4P\x99\x1d\xa0\x90}\xd4\xc8\xc2\x8b\xc6E\x07\x8c\xe0\x82t\x8f\x90V?\xfb\x7f<#\xd9s\xc3\xa3\xf7\xea\x95v1\xfa$\x9eQ\x93\xfd\xe7\x1c"A\xa8\x01\xeah\x99\xc1\xa1\x8c\xaa\'\xecqZ!\x8e\xc9\xa6\xc7~\xff\x9cy\x8b\xc2\xa5\xdb\xa6\xf8DM\xffq\x1f\xe5\x10J\xfe\'\xa3\x0c\x0b\x8b"(&eayG\x15\xcc\xea\xf1\x86\xbc\x04\xd6\xb5^\x0c\xdf5W\xd2\x8c\xe8=\x99\x07\xee\xb9\x7fA)\x87NE'
04c940f89e4b2fb757056cc643161db1753c9d16fb614d0b7e4a77a6bbe3fccb57d4af9b218fc98cfb3d26ed98d64b285c6440609479eecdfbc87e21b6edbdead947d6f09a34961aa50f8ca6e726ff0a73844e16f67cdf8ccff1339ab76923d5678c7e0

In [97]:
embed_lsb("lsb_w_ECC.png", ciphertext.hex())

In [100]:
extract_lsb("lsb_w_ECC.png")

'04c940f89e4b2fb757056cc643161db1753c9d16fb614d0b7e4a77a6bbe3fccb57d4af9b218fc98cfb3d26ed98d64b285c6440609479eecdfbc87e21b6edbdead947d6f09a34961aa50f8ca6e726ff0a73844e16f67cdf8ccff1339ab76923d5678c7e004d8461d3af6bea7b89bf0fbfa5b2214aeb43b288919e7c565463bb677b02aeb2619763622a4707cbac1460e3b8d118b243890229f450991da0907dd4c8c28bc645078ce082748f90563ffb7f3c23d973c3a3f7ea957631fa249e5193fde71c2241a801ea6899c1a18caa27ec715a218ec9a6c77eff9c798bc2a5dba6f8444dff711fe5104afe27a30c0b8b2228266561794715cceaf186bc04d6b55e0cdf3557d28ce83d9907eeb97f4129874e45'

In [101]:
calculate_psnr_2("lsb_w_ECC.png")

PSNR: 73.72682259672885


73.72682259672885

# LSB with LZW

## LZW

In [132]:
import math

def lzw_compress(uncompressed):
    """Compress the string using LZW algorithm."""
    dict_size = 256
    dictionary = {chr(i): i for i in range(dict_size)}

    w = ""
    compressed_data = []

    for c in uncompressed:
        wc = w + c
        if wc in dictionary:
            w = wc
        else:
            compressed_data.append(dictionary[w])
            dictionary[wc] = dict_size
            dict_size += 1
            w = c

    if w:
        compressed_data.append(dictionary[w])

    return compressed_data, dict_size

def lzw_decompress(compressed):
    """Decompress the LZW compressed data."""
    dict_size = 256
    dictionary = {i: chr(i) for i in range(dict_size)}

    w = chr(compressed.pop(0))
    decompressed_data = w

    for k in compressed:
        if k in dictionary:
            entry = dictionary[k]
        elif k == dict_size:
            entry = w + w[0]
        else:
            raise ValueError(f"Bad compressed k: {k} (dictionary size: {dict_size})")

        decompressed_data += entry
        dictionary[dict_size] = w + entry[0]
        dict_size += 1
        w = entry

    return decompressed_data

# Function to calculate size in bytes or bits
def calculate_size(data, dict_size=None):
    if isinstance(data, str):
        # Size in bytes and bits for string (1 byte = 8 bits for each character)
        size_in_bytes = len(data.encode('utf-8'))
        size_in_bits = size_in_bytes * 8
        return size_in_bytes, size_in_bits  # Size in bytes and bits
    elif isinstance(data, list):
        # Calculate the bit size for compressed data (variable bit-length encoding)
        if dict_size is None:
            raise ValueError("Dictionary size needed for proper bit calculation!")
        bits_needed = math.ceil(math.log2(dict_size))  # Number of bits required
        total_bits = len(data) * bits_needed
        return total_bits // 8, total_bits  # Returns size in bytes and bits

# Example usage:
input_string = CONTENT
print(f"Original String: {input_string}")

# Compress the input string
compressed, dict_size = lzw_compress(input_string)
decompressed = lzw_decompress(compressed)

# Sizes
original_size_bytes, original_size_bits = calculate_size(input_string)  # Original size in bytes and bits
compressed_size_bytes, compressed_size_bits = calculate_size(compressed, dict_size)  # Compressed size in bytes and bits
decompressed_size_bytes, decompressed_size_bits = calculate_size(decompressed)  # Decompressed size in bytes and bits

print(f"\nOriginal String Size: {original_size_bytes} bytes ({original_size_bits} bits)")
print(f"Compressed Data Size: {compressed_size_bytes} bytes ({compressed_size_bits} bits)")
print(f"Decompressed String Size: {decompressed_size_bytes} bytes ({decompressed_size_bits} bits)")

# Check if decompressed string matches the original string
if decompressed == input_string:
    print("Decompression was successful!")
    print()
    print(f"Decompressed String: {decompressed}")
else:
    print("Decompression failed.")

Original String: 3348610401970005#christofer*derian*budianto#tegal#1997-03-04#laki-laki#b#jl.*pala*22*no.*30#005#017#mejasem*tengah#kramat#katholik#belum*kawin#pelajar/mahasiswa#wni#seumur*hidup

Original String Size: 177 bytes (1416 bits)
Compressed Data Size: 166 bytes (1332 bits)
Decompressed String Size: 177 bytes (1416 bits)
Decompression was successful!

Decompressed String: 3348610401970005#christofer*derian*budianto#tegal#1997-03-04#laki-laki#b#jl.*pala*22*no.*30#005#017#mejasem*tengah#kramat#katholik#belum*kawin#pelajar/mahasiswa#wni#seumur*hidup


In [140]:
print(compressed)

[56, 54, 49, 48, 52, 48, 49, 57, 55, 48, 268, 53, 35, 99, 104, 114, 105, 115, 116, 111, 102, 101, 114, 42, 100, 280, 105, 97, 110, 42, 98, 117, 100, 285, 110, 277, 35, 116, 101, 103, 97, 108, 35, 265, 266, 45, 48, 51, 304, 52, 35, 108, 97, 107, 105, 45, 310, 312, 35, 98, 35, 106, 108, 46, 42, 112, 299, 97, 42, 50, 50, 42, 110, 111, 322, 51, 48, 35, 269, 336, 49, 55, 35, 109, 101, 106, 97, 115, 101, 109, 42, 296, 110, 298, 104, 35, 107, 114, 97, 109, 97, 116, 354, 359, 104, 111, 108, 105, 107, 317, 101, 108, 117, 348, 107, 97, 119, 105, 110, 35, 112, 369, 97, 344, 114, 47, 358, 104, 345, 275, 119, 97, 35, 119, 110, 105, 35, 346, 371, 117, 281, 104, 105, 100, 117, 112]


In [119]:
compressed_str = ','.join(map(str, compressed))

compressed_str

'51,52,56,54,49,48,52,48,49,57,55,48,268,53,35,99,104,114,105,115,116,111,102,101,114,42,100,280,105,97,110,42,98,117,100,285,110,277,35,116,101,103,97,108,35,265,266,45,48,51,304,52,35,108,97,107,105,45,310,312,35,98,35,106,108,46,42,112,299,97,42,50,50,42,110,111,322,51,48,35,269,336,49,55,35,109,101,106,97,115,101,109,42,296,110,298,104,35,107,114,97,109,97,116,354,359,104,111,108,105,107,317,101,108,117,348,107,97,119,105,110,35,112,369,97,344,114,47,358,104,345,275,119,97,35,119,110,105,35,346,371,117,281,104,105,100,117,112'

In [120]:
embed_lsb("lsb_w_LZW.png", compressed_str)

In [121]:
extracted_lzw_from_lsb = extract_lsb("lsb_w_LZW.png")
extracted_lzw_from_lsb

'51,52,56,54,49,48,52,48,49,57,55,48,268,53,35,99,104,114,105,115,116,111,102,101,114,42,100,280,105,97,110,42,98,117,100,285,110,277,35,116,101,103,97,108,35,265,266,45,48,51,304,52,35,108,97,107,105,45,310,312,35,98,35,106,108,46,42,112,299,97,42,50,50,42,110,111,322,51,48,35,269,336,49,55,35,109,101,106,97,115,101,109,42,296,110,298,104,35,107,114,97,109,97,116,354,359,104,111,108,105,107,317,101,108,117,348,107,97,119,105,110,35,112,369,97,344,114,47,358,104,345,275,119,97,35,119,110,105,35,346,371,117,281,104,105,100,117,112'

In [170]:
calculate_psnr_2("lsb_w_LZW.png")

PSNR: 73.77314241018492


73.77314241018492

In [151]:
extracted_compressed_data = list(map(int, extracted_lzw_from_lsb.split(',')))
extracted_compressed_data

[51,
 52,
 56,
 54,
 49,
 48,
 52,
 48,
 49,
 57,
 55,
 48,
 268,
 53,
 35,
 99,
 104,
 114,
 105,
 115,
 116,
 111,
 102,
 101,
 114,
 42,
 100,
 280,
 105,
 97,
 110,
 42,
 98,
 117,
 100,
 285,
 110,
 277,
 35,
 116,
 101,
 103,
 97,
 108,
 35,
 265,
 266,
 45,
 48,
 51,
 304,
 52,
 35,
 108,
 97,
 107,
 105,
 45,
 310,
 312,
 35,
 98,
 35,
 106,
 108,
 46,
 42,
 112,
 299,
 97,
 42,
 50,
 50,
 42,
 110,
 111,
 322,
 51,
 48,
 35,
 269,
 336,
 49,
 55,
 35,
 109,
 101,
 106,
 97,
 115,
 101,
 109,
 42,
 296,
 110,
 298,
 104,
 35,
 107,
 114,
 97,
 109,
 97,
 116,
 354,
 359,
 104,
 111,
 108,
 105,
 107,
 317,
 101,
 108,
 117,
 348,
 107,
 97,
 119,
 105,
 110,
 35,
 112,
 369,
 97,
 344,
 114,
 47,
 358,
 104,
 345,
 275,
 119,
 97,
 35,
 119,
 110,
 105,
 35,
 346,
 371,
 117,
 281,
 104,
 105,
 100,
 117,
 112]

In [155]:
decompressed = lzw_decompress(extracted_compressed_data)
decompressed

ValueError: Bad compressed k: 268 (dictionary size: 267)

only decompress sequenced after compress

## LZW Package

In [159]:
!git clone https://github.com/joeatwork/python-lzw

fatal: destination path 'python-lzw' already exists and is not an empty directory.


In [160]:
!cd /content/python-lzw && mv lzw/ /content/ && python setup.py install

running install
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https

In [161]:
import lzw

# Example text to compress (can be any string)
original_text = CONTENT

# Convert the original text into bytes (UTF-8 encoding)
original_bytes = original_text.encode('utf-8')

# Part 1: Compression (in-memory)
compressed = lzw.compress(original_bytes)

# Convert compressed generator to a list to see its contents
compressed_list = list(compressed)

# Part 2: Decompression (in-memory)
decompressed_chunks = lzw.decompress(compressed_list)

# Join the decompressed chunks into a single byte string
decompressed_bytes = b"".join(decompressed_chunks)

# Convert decompressed bytes back to a string
decompressed_text = decompressed_bytes.decode('utf-8')

# Print each variable
print("Original Text (String):", original_text)
print("Original Bytes:", original_bytes)
print("Compressed List (Byte Chunks):", compressed_list)
print("Decompressed Bytes:", decompressed_bytes)
print("Decompressed Text (String):", decompressed_text)

# Part 3: Print sizes and results for comparison
original_size = len(original_bytes)
compressed_size = len(b"".join(compressed_list))  # Join to get compressed byte size
decompressed_size = len(decompressed_bytes)

# Print the sizes in bytes
print(f"\nOriginal size: {original_size} bytes")
print(f"Compressed size: {compressed_size} bytes")
print(f"Decompressed size: {decompressed_size} bytes")

# Check if decompression was successful
if decompressed_text == original_text:
    print("Decompression was successful!")
else:
    print("Decompression failed.")

Original Text (String): 3348610401970005#christofer*derian*budianto#tegal#1997-03-04#laki-laki#b#jl.*pala*22*no.*30#005#017#mejasem*tengah#kramat#katholik#belum*kawin#pelajar/mahasiswa#wni#seumur*hidup
Original Bytes: b'3348610401970005#christofer*derian*budianto#tegal#1997-03-04#laki-laki#b#jl.*pala*22*no.*30#005#017#mejasem*tengah#kramat#katholik#belum*kawin#pelajar/mahasiswa#wni#seumur*hidup'
Compressed List (Byte Chunks): [b'\x19', b'\x8c', b'\xc6', b'\x83', b'\x81', b'\xb0', b'\xc4', b'`', b'4', b'\x18', b'\x0c', b'G', b'#', b'q', b'\x84', b'8', b'j', b'#', b'1', b'\x9a', b'\x0e', b'F', b'\x93', b'\x99', b'\xd0', b'\xde', b'f', b'2', b'\x9c', b'\x85', b'F', b'H', b'\xd1', b'\xa4', b'\xc2', b'n', b'\x15', b'\x18', b'\x8e', b'\xa6', b'H', b'\xf9', b'\xba', b'.', b'#', b':', b'\x19', b'L', b'\xe6', b'\x13', b'`', b'\x8e', b'\x17', b'\x0c', b'\x16', b'\x8c', b'\x06', b's', b'!', b'\xa0', b'\x8c', b'\xd8', b'a', b'5', b'\x9a', b'E', b'\xb3', b'\x89', b'\xd0', b'\x8c', b'\xc4', b'#', b'

In [176]:
compressed

<generator object BitPacker.pack at 0x7f2dce3b7300>

In [166]:
compressed_list

[b'\x19',
 b'\x8c',
 b'\xc6',
 b'\x83',
 b'\x81',
 b'\xb0',
 b'\xc4',
 b'`',
 b'4',
 b'\x18',
 b'\x0c',
 b'G',
 b'#',
 b'q',
 b'\x84',
 b'8',
 b'j',
 b'#',
 b'1',
 b'\x9a',
 b'\x0e',
 b'F',
 b'\x93',
 b'\x99',
 b'\xd0',
 b'\xde',
 b'f',
 b'2',
 b'\x9c',
 b'\x85',
 b'F',
 b'H',
 b'\xd1',
 b'\xa4',
 b'\xc2',
 b'n',
 b'\x15',
 b'\x18',
 b'\x8e',
 b'\xa6',
 b'H',
 b'\xf9',
 b'\xba',
 b'.',
 b'#',
 b':',
 b'\x19',
 b'L',
 b'\xe6',
 b'\x13',
 b'`',
 b'\x8e',
 b'\x17',
 b'\x0c',
 b'\x16',
 b'\x8c',
 b'\x06',
 b's',
 b'!',
 b'\xa0',
 b'\x8c',
 b'\xd8',
 b'a',
 b'5',
 b'\x9a',
 b'E',
 b'\xb3',
 b'\x89',
 b'\xd0',
 b'\x8c',
 b'\xc4',
 b'#',
 b'5',
 b'\x1b',
 b'\x05',
 b'\xc2',
 b'\xa3',
 b'\x84',
 b'\xb4',
 b'\xc2',
 b'*',
 b'\x19',
 b'\x0c',
 b'\x85',
 b'F',
 b'\xe3',
 b'}',
 b'\x10',
 b'f',
 b'0',
 b'\x11',
 b'\xc3',
 b'\xea',
 b'C',
 b'\x11',
 b'\xb8',
 b'\x8c',
 b'\xda',
 b'e',
 b'5',
 b'\x18',
 b'N',
 b'f',
 b'S',
 b'h',
 b'\xaa',
 b'T',
 b'n',
 b'\x96',
 b'\x1a',
 b'\x04',
 b'f',
 b'\xb3',

In [182]:
type(compressed_list)

list

In [177]:
embed_lsb("lsb_w_LZW_package.png", compressed_list)

In [178]:
extracted_data = extract_lsb("lsb_w_LZW_package.png")
extracted_data

'\x19\x8cÆ\x83\x81°Ä`4\x18\x0cG#q\x848j#1\x9a\x0eF\x93\x99ÐÞf2\x9c\x85FHÑ¤Ân\x15\x18\x8e¦Hùº.#:\x19Læ\x13`\x8e\x17\x0c\x16\x8c\x06s!\xa0\x8cØa5\x9aE³\x89Ð\x8cÄ#5\x1b\x05Â£\x84´Â*\x19\x0c\x85Fã}\x10f0\x11ÃêC\x11¸\x8cÚe5\x18NfShªTn\x96\x1a\x04f³\x91\x84Úa:Y-&\x83y°Òk\x9f\x99M\x87Zñ¬Âw4\x9b\x84g\x0b\x99\x86´r\x17Ú\r\x15¸©ÜÂ#;\x9b\x8d":åÔë\x1b4\x1aL\x87S\x84\x00'

In [183]:
output_list = [repr(byte.encode('latin1')) for byte in extracted_data]

# Print in the requested format
formatted_output = "[\n" + ",\n ".join(output_list) + "\n]"

print(formatted_output)

[
b'\x19',
 b'\x8c',
 b'\xc6',
 b'\x83',
 b'\x81',
 b'\xb0',
 b'\xc4',
 b'`',
 b'4',
 b'\x18',
 b'\x0c',
 b'G',
 b'#',
 b'q',
 b'\x84',
 b'8',
 b'j',
 b'#',
 b'1',
 b'\x9a',
 b'\x0e',
 b'F',
 b'\x93',
 b'\x99',
 b'\xd0',
 b'\xde',
 b'f',
 b'2',
 b'\x9c',
 b'\x85',
 b'F',
 b'H',
 b'\xd1',
 b'\xa4',
 b'\xc2',
 b'n',
 b'\x15',
 b'\x18',
 b'\x8e',
 b'\xa6',
 b'H',
 b'\xf9',
 b'\xba',
 b'.',
 b'#',
 b':',
 b'\x19',
 b'L',
 b'\xe6',
 b'\x13',
 b'`',
 b'\x8e',
 b'\x17',
 b'\x0c',
 b'\x16',
 b'\x8c',
 b'\x06',
 b's',
 b'!',
 b'\xa0',
 b'\x8c',
 b'\xd8',
 b'a',
 b'5',
 b'\x9a',
 b'E',
 b'\xb3',
 b'\x89',
 b'\xd0',
 b'\x8c',
 b'\xc4',
 b'#',
 b'5',
 b'\x1b',
 b'\x05',
 b'\xc2',
 b'\xa3',
 b'\x84',
 b'\xb4',
 b'\xc2',
 b'*',
 b'\x19',
 b'\x0c',
 b'\x85',
 b'F',
 b'\xe3',
 b'}',
 b'\x10',
 b'f',
 b'0',
 b'\x11',
 b'\xc3',
 b'\xea',
 b'C',
 b'\x11',
 b'\xb8',
 b'\x8c',
 b'\xda',
 b'e',
 b'5',
 b'\x18',
 b'N',
 b'f',
 b'S',
 b'h',
 b'\xaa',
 b'T',
 b'n',
 b'\x96',
 b'\x1a',
 b'\x04',
 b'f',
 b'\xb3'

In [185]:
type(formatted_output)

str

In [186]:
byte_list = [bytes([ord(char)]) for char in extracted_data]

# Print the byte list in the required format
print(byte_list)

[b'\x19', b'\x8c', b'\xc6', b'\x83', b'\x81', b'\xb0', b'\xc4', b'`', b'4', b'\x18', b'\x0c', b'G', b'#', b'q', b'\x84', b'8', b'j', b'#', b'1', b'\x9a', b'\x0e', b'F', b'\x93', b'\x99', b'\xd0', b'\xde', b'f', b'2', b'\x9c', b'\x85', b'F', b'H', b'\xd1', b'\xa4', b'\xc2', b'n', b'\x15', b'\x18', b'\x8e', b'\xa6', b'H', b'\xf9', b'\xba', b'.', b'#', b':', b'\x19', b'L', b'\xe6', b'\x13', b'`', b'\x8e', b'\x17', b'\x0c', b'\x16', b'\x8c', b'\x06', b's', b'!', b'\xa0', b'\x8c', b'\xd8', b'a', b'5', b'\x9a', b'E', b'\xb3', b'\x89', b'\xd0', b'\x8c', b'\xc4', b'#', b'5', b'\x1b', b'\x05', b'\xc2', b'\xa3', b'\x84', b'\xb4', b'\xc2', b'*', b'\x19', b'\x0c', b'\x85', b'F', b'\xe3', b'}', b'\x10', b'f', b'0', b'\x11', b'\xc3', b'\xea', b'C', b'\x11', b'\xb8', b'\x8c', b'\xda', b'e', b'5', b'\x18', b'N', b'f', b'S', b'h', b'\xaa', b'T', b'n', b'\x96', b'\x1a', b'\x04', b'f', b'\xb3', b'\x91', b'\x84', b'\xda', b'a', b':', b'Y', b'-', b'&', b'\x83', b'y', b'\xb0', b'\xd2', b'k', b'\x9f', b'\x99

In [188]:
type(byte_list)

list

In [190]:
calculate_psnr_2("lsb_w_LZW_package.png")

PSNR: 78.72417421832482


78.72417421832482

In [189]:
# Part 2: Decompression (in-memory)
decompressed_chunks = lzw.decompress(byte_list)

# Join the decompressed chunks into a single byte string
decompressed_bytes = b"".join(decompressed_chunks)

# Convert decompressed bytes back to a string
decompressed_text = decompressed_bytes.decode('utf-8')

print("Decompressed Text (String):", decompressed_text)

Decompressed Text (String): 3348610401970005#christofer*derian*budianto#tegal#1997-03-04#laki-laki#b#jl.*pala*22*no.*30#005#017#mejasem*tengah#kramat#katholik#belum*kawin#pelajar/mahasiswa#wni#seumur*hidup


# LSB With Huffman Encoding

In [192]:
import heapq
from collections import Counter

# Class for Huffman Tree Nodes
class HuffmanNode:
    def __init__(self, char=None, freq=0, left=None, right=None):
        self.char = char
        self.freq = freq
        self.left = left
        self.right = right

    def __lt__(self, other):
        return self.freq < other.freq

# Function to build the Huffman Tree
def build_huffman_tree(frequency):
    heap = [HuffmanNode(char, freq) for char, freq in frequency.items()]
    heapq.heapify(heap)

    while len(heap) > 1:
        node1 = heapq.heappop(heap)
        node2 = heapq.heappop(heap)
        merged = HuffmanNode(freq=node1.freq + node2.freq, left=node1, right=node2)
        heapq.heappush(heap, merged)

    return heap[0]  # Return the root of the tree

# Function to generate Huffman codes from the Huffman Tree
def generate_huffman_codes(node, current_code="", huffman_codes={}):
    if node is None:
        return

    if node.char is not None:
        huffman_codes[node.char] = current_code

    generate_huffman_codes(node.left, current_code + "0", huffman_codes)
    generate_huffman_codes(node.right, current_code + "1", huffman_codes)

    return huffman_codes

# Function to compress the data using Huffman Encoding
def huffman_compress(data):
    # Count the frequency of each byte
    frequency = Counter(data)

    # Build Huffman Tree
    huffman_tree = build_huffman_tree(frequency)

    # Generate Huffman Codes
    huffman_codes = generate_huffman_codes(huffman_tree)

    # Encode data into its binary representation
    encoded_data = "".join(huffman_codes[byte] for byte in data)

    # Pad the encoded data to make its length a multiple of 8
    padding = 8 - len(encoded_data) % 8
    encoded_data += "0" * padding
    padding_info = "{0:08b}".format(padding)

    # Convert binary data to bytes
    compressed_data = bytearray()
    compressed_data.append(int(padding_info, 2))  # First byte stores the padding information
    for i in range(0, len(encoded_data), 8):
        byte = encoded_data[i:i+8]
        compressed_data.append(int(byte, 2))

    return compressed_data, huffman_tree

# Function to decompress the data using Huffman Encoding
def huffman_decompress(compressed_data, huffman_tree):
    # Convert the compressed data to binary string
    encoded_data = ""
    for byte in compressed_data[1:]:
        encoded_data += "{0:08b}".format(byte)

    # Get the padding information from the first byte
    padding = compressed_data[0]
    encoded_data = encoded_data[:-padding]  # Remove padding

    # Traverse the Huffman tree to decode the binary string
    decoded_data = bytearray()
    node = huffman_tree
    for bit in encoded_data:
        if bit == "0":
            node = node.left
        else:
            node = node.right

        if node.left is None and node.right is None:  # Leaf node
            decoded_data.append(node.char)
            node = huffman_tree

    return decoded_data


# Example Usage of Huffman Compression/Decompression

# Step 1: The original plaintext data (as a byte string)
plaintext = CONTENT.encode()

# Print the original data
print("Original Data (as bytes):", plaintext)
print("Original Data (as string):", plaintext.decode())

# Step 2: Compress the plaintext data using Huffman encoding
compressed_data, huffman_tree = huffman_compress(plaintext)
print("Compressed Data (in bytes):", compressed_data)

# Step 3: Decompress the compressed data back to the original plaintext
decompressed_data = huffman_decompress(compressed_data, huffman_tree)
print("Decompressed Data (as bytes):", decompressed_data)
print("Decompressed Data (as string):", decompressed_data.decode())

# Step 4: Verify that the decompressed data matches the original data
if decompressed_data == plaintext:
    print("Decompression was successful!")
else:
    print("Decompression failed.")


Original Data (as bytes): b'3348610401970005#christofer*derian*budianto#tegal#1997-03-04#laki-laki#b#jl.*pala*22*no.*30#005#017#mejasem*tengah#kramat#katholik#belum*kawin#pelajar/mahasiswa#wni#seumur*hidup'
Original Data (as string): 3348610401970005#christofer*derian*budianto#tegal#1997-03-04#laki-laki#b#jl.*pala*22*no.*30#005#017#mejasem*tengah#kramat#katholik#belum*kawin#pelajar/mahasiswa#wni#seumur*hidup
Compressed Data (in bytes): bytearray(b'\x02\xdfy>\xa8\x7f\x12G\xf7S\x11\x1e\xfb\xedYu\xcf\xc6)\x91\xd52\xe6\xc3\xa3usi\xf9\xd3OD;\xfb\xae\xa6\xf8\xde\xf8\x93\x80\xe1\xef\x03\x87\xed9\x90\xa2;\x10#\xf3\xe0\xdb\xca#\xbctG\xbe\x8f\xe9\xea\xa3%iQ\x9a[\xd1Wc&\xa6}\x835x\x0f\x8e\xd1\x01\xb5\x1c\x19^\xdd\xb2\x01d\xe5\x85J\x8a\xddr\x9e\x96\xcf\xcbF\xd5\xb9\x15>\xad\xd8')
Decompressed Data (as bytes): bytearray(b'3348610401970005#christofer*derian*budianto#tegal#1997-03-04#laki-laki#b#jl.*pala*22*no.*30#005#017#mejasem*tengah#kramat#katholik#belum*kawin#pelajar/mahasiswa#wni#seumur*hidup')

In [196]:
compressed_data.hex()

'02df793ea87f1247f753111efbed5975cfc62991d532e6c3a3757369f9d34f443bfbaea6f8def89380e1ef0387ed3990a23b1023f3e0dbca23bc7447be8fe9eaa32569519a5bd1576326a67d8335780f8ed101b51c195eddb20164e5854a8add729e96cfcb46d5b9153eadd8'

In [197]:
embed_lsb("lsb_w_Huffman.png", compressed_data.hex())

In [198]:
extracted_data = extract_lsb("lsb_w_Huffman.png")
extracted_data

'02df793ea87f1247f753111efbed5975cfc62991d532e6c3a3757369f9d34f443bfbaea6f8def89380e1ef0387ed3990a23b1023f3e0dbca23bc7447be8fe9eaa32569519a5bd1576326a67d8335780f8ed101b51c195eddb20164e5854a8add729e96cfcb46d5b9153eadd8'

In [199]:
calculate_psnr_2("lsb_w_Huffman.png")

PSNR: 77.81371174500211


77.81371174500211

In [201]:
byte_array = bytearray.fromhex(extracted_data)

# Print the bytearray
print(byte_array)

bytearray(b'\x02\xdfy>\xa8\x7f\x12G\xf7S\x11\x1e\xfb\xedYu\xcf\xc6)\x91\xd52\xe6\xc3\xa3usi\xf9\xd3OD;\xfb\xae\xa6\xf8\xde\xf8\x93\x80\xe1\xef\x03\x87\xed9\x90\xa2;\x10#\xf3\xe0\xdb\xca#\xbctG\xbe\x8f\xe9\xea\xa3%iQ\x9a[\xd1Wc&\xa6}\x835x\x0f\x8e\xd1\x01\xb5\x1c\x19^\xdd\xb2\x01d\xe5\x85J\x8a\xddr\x9e\x96\xcf\xcbF\xd5\xb9\x15>\xad\xd8')


In [202]:
decompressed_data = huffman_decompress(byte_array, huffman_tree)
print("Decompressed Data (as bytes):", decompressed_data)

Decompressed Data (as bytes): bytearray(b'3348610401970005#christofer*derian*budianto#tegal#1997-03-04#laki-laki#b#jl.*pala*22*no.*30#005#017#mejasem*tengah#kramat#katholik#belum*kawin#pelajar/mahasiswa#wni#seumur*hidup')


In [203]:
print("Decompressed Data (as string):", decompressed_data.decode())

Decompressed Data (as string): 3348610401970005#christofer*derian*budianto#tegal#1997-03-04#laki-laki#b#jl.*pala*22*no.*30#005#017#mejasem*tengah#kramat#katholik#belum*kawin#pelajar/mahasiswa#wni#seumur*hidup
